## Run the current pipeline snippet (no new logic)

This notebook cell calls the same pipeline function used by `python/run_all.py` for AcceptCount stored-procedure execution.

- Function called: `sql_operations.accept_count_operation.run_accept_count(...)`
- Behavior: executes `USP_SystemAlert_AcceptCountProcedure` and writes raw result-set CSVs to `data/refresh/`.

This is a direct pipeline invocation for quick validation in notebook form.

In [1]:

import platform, sys
print(platform.machine())
print(sys.executable)

x86_64
/Users/starsrain/2025_concord/py_loonie_perf_reporting/.venv/bin/python


In [2]:
from pathlib import Path
import sys
import pandas as pd

# Ensure repo root is importable when notebook runs from notebooks/
repo_root = Path.cwd()
if (repo_root / "python").exists() is False:
    repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Use existing pipeline functions (same path as run_all.py)
from sql_operations.accept_count_operation import run_accept_count, summarize_accept_count_from_proc

DATE_RANGE = 3
TIME_RANGE = 4

# 1) Run stored procedure through normal pipeline path
written_files = run_accept_count(date_range=DATE_RANGE, time_range=TIME_RANGE)
print("Pipeline wrote:")
for p in written_files:
    print(" -", p)

# 2) Run pipeline summary helper used by orchestration
summary = summarize_accept_count_from_proc(date_range=DATE_RANGE, time_range=TIME_RANGE)
print("\nPipeline summary:", summary)

# 3) Optional peek at the by-provider CSV from pipeline output
by_provider_path = next((p for p in written_files if p.name == "accept_count_by_provider.csv"), None)
if by_provider_path is not None and by_provider_path.exists():
    by_provider_df = pd.read_csv(by_provider_path)
    display(by_provider_df.sort_values("ApplicationCount", ascending=False).head(20))

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib '/opt/homebrew/lib/libmsodbcsql.18.dylib' : file not found (0) (SQLDriverConnect)")